In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")


In [ ]:
train = pd.read_csv ("/kaggle/input/digit-recognizer/train.csv")
test = pd.read_csv ("/kaggle/input/digit-recognizer/test.csv")

In [ ]:
train.shape, test.shape

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
# seperate pixels features as x_train and target feature as y_train
Y_train = train["label"]
X_train = train.drop(labels = ["label"], axis =1)

In [ ]:
Y_train.value_counts()


In [ ]:
for i in range(0,4):
    img = X_train.iloc[i].to_numpy()
    img = img.reshape((28,28))
    plt.imshow(img, cmap = "gray")
    plt.title(train.iloc[0,0])
    plt.axis("off")
    plt.show()

In [ ]:
# Normalization
X_train = X_train /255
test = test/255

In [ ]:
# Reshape
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)


In [ ]:
X_train.shape, test.shape

In [ ]:
from keras.utils.np_utils import to_categorical

In [ ]:
Y_train = to_categorical(Y_train, num_classes = 10)

In [ ]:
Y_train = Y_train.astype("int8")

In [ ]:
Y_train[0:4]

In [ ]:
# seperate dataset with train_test_split
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.20, random_state=42)

In [ ]:
x_train.shape, x_val.shape, y_train.shape, y_val.shape

In [ ]:
?padding

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf
from keras.layers import Flatten, Dense, Reshape

model = Sequential()




model.add(Conv2D(filters = 96, kernel_size = (11,11),padding = 'Valid', 
                 activation ='relu',strides=(4,4), input_shape = ((28,28,1))))
#model.add(Reshape((227,227,3), input_shape=(28,28,1)))

model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))
model.add(Dropout(0.25))
#
model.add(Conv2D(filters = 96, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))
model.add(Dropout(0.25))
#
model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
#model.add(MaxPool2D(pool_size=(2,2), strides=(1,1)))
model.add(Dropout(0.25))
#
model.add(Conv2D(filters = 384, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
#model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 384, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
#model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))
model.add(Dropout(0.25))

# fully connected
model.add(Flatten())
model.add(Dense(9216, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(4096, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(4096, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [ ]:
# Define the optimizer
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

In [ ]:
# Compile the model
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
epochs = 50  # for better result increase the epochs
batch_size = 512

In [ ]:
# data augmentation
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # dimesion reduction
        rotation_range=0.5,  # randomly rotate images in the range 5 degrees
        zoom_range = 0.5, # Randomly zoom image 5%
        width_shift_range=0.5,  # randomly shift images horizontally 5%
        height_shift_range=0.5,  # randomly shift images vertically 5%
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(X_train)

In [ ]:
# Fit the model method 1
# history = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size), #                             epochs = epochs, validation_data = (x_val,y_val), steps_per_epoch=x_train.shape[0] // batch_size)

In [ ]:
model.fit(x_train, y_train, batch_size=512, epochs=100)

In [ ]:
# Plot the loss and accuracy curves for training and validation 
plt.plot(history.history['val_loss'], color='b', label="validation loss")
plt.title("Test Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
# confusion matrix
import seaborn as sns
# Predict the values from the validation dataset
Y_pred = model.predict(x_val)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_val,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
f,ax = plt.subplots(figsize=(8, 8))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
test_result = model.predict(test)


results = np.argmax(test_result,axis = 1) 

results = pd.Series(results,name="Label")


submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("submiss_CNN_LeNet_v2.csv",index=False)